In [89]:
import numpy as np

# Concatena gli array
features = np.load('/kaggle/input/scenicornot-counts/array_immagini96_completo11conti.npy')
votes = np.load('/kaggle/input/scenicornot-counts/array_voti_completo11conti.npy')

In [45]:
# calcolo delle medie

means = []

for array in votes:
    means.append(np.sum((array*[1,2,3,4,5,6,7,8,9,10])/np.sum(array)))
    

means = np.array(means)

In [46]:
tot_votes = np.sum(votes, axis=1)
max(tot_votes)

158

In [49]:
# Nuovo dataset in cui con probabilità proporzionale al numero di voti seleziono un'immagine con reimmissione 
# e la duplico 

# poi di questo nuovo dataset ogni immagine viene copiata tante volte quanti sono i voti 
# su cui ha un punteggio non zero e assegno ad ogni copia la lable del diverso punteggio

#es, foto che ha [0,2,3,0,0,0,1,0,0]: l'immagine è copiata 3 volte e alle 3 copie sono assegnate label 1,2,6

def create_random_copies(images,votes, probabilities, N):
    indices = np.arange(len(images))
    selected_indices = np.random.choice(indices, size=N, replace=True, p=probabilities)
    selected_images = images[selected_indices]
    selected_votes  = votes[selected_indices]
    return selected_images, selected_votes

N = len(features)  # Numero di copie da creare

# Calcola le probabilità normalizzate
prob = tot_votes / np.sum(tot_votes)

# Crea le copie di immagini selezionate casualmente
copied_images, copied_votes = create_random_copies(features,votes, prob, N)

In [50]:
features=np.concatenate((features,copied_images))
votes=np.concatenate((votes,copied_votes))

In [42]:
features.shape

(7266, 96, 96, 3)

In [51]:
distr=votes/np.sum(votes,axis=1)[:,np.newaxis]
len(distr)

7266

In [52]:
features_def =[]
target_def  =[]

for image, j in zip(features, range(len(features)-1)):
    
    for i in range(10):
        
        if distr[j,i]>0.01:
            
            features_def.append(image)
            target_def.append(i+1)

In [54]:
features_def =np.array(features_def)
target_def  =np.array(target_def)

features_def.shape

(41975, 96, 96, 3)

In [55]:
features_def  = features_def/255

In [57]:
import tensorflow as tf
tf.random.set_seed(582) #per riproducibilità

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [60]:
from sklearn.model_selection import train_test_split

trainval_features, test_features, trainval_target, test_target = train_test_split(features_def, target_def, test_size=0.2)

# controllo training set
print('TRAINING SET:\t',trainval_features.shape)

# controllo test set
print('TEST SET:\t',test_features.shape)

# validazione

indices = np.arange(len(trainval_features))

train_features, val_features, train_target, val_target = train_test_split(trainval_features, trainval_target, test_size=0.1)

# controllo training set
print('TRAINING SET:\t',train_features.shape)

# controllo validation set
print('VALIDATION SET:\t',val_features.shape)

TRAINING SET:	 (33580, 96, 96, 3)
TEST SET:	 (8395, 96, 96, 3)
TRAINING SET:	 (30222, 96, 96, 3)
VALIDATION SET:	 (3358, 96, 96, 3)


In [61]:
del features_def, trainval_features

In [132]:
#import per keras

from tensorflow import keras
from tensorflow.keras import Model, Input,regularizers

# import per i layers

from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPool2D, Reshape, AvgPool2D, Dropout

#eliminazione backend

keras.backend.clear_session()

In [133]:
#definizione del modello, inputs (96x96x3)

inputs = Input(shape=(96,96,3)) 

#C1
x = Conv2D(filters=6, kernel_size=(5,5), padding='same', strides=(1,1), activation='relu')(inputs)

#S2
x = MaxPool2D(pool_size=(2,2), strides=(2,2),  padding='valid')(x)

#C3
x = Conv2D(filters=16, kernel_size=(5,5), padding='valid', strides=(1,1), activation='relu')(x)

#S4
x = AvgPool2D(pool_size=(2,2), strides=(2,2),  padding='valid')(x)

#C5 con regolarizzatore l2
x = Conv2D(filters=256, kernel_size=(7,7), padding='valid', strides=(1,1), activation='relu',kernel_regularizer=keras.regularizers.l2(1e-3))(x)

# flatten 
x = Flatten()(x)

#F6 con regolarizzatore l1
x = Dense(84, activation='relu', kernel_regularizer=keras.regularizers.l1(1e-3))(x)

#D7
dropout_rate = 0.1  # Tasso di dropout desiderato
x = Dropout(rate=dropout_rate)(x)

# Funzione di trasformazione lineare tra 1 e 10
def custom_activation(x):
    min_value = 1  # Valore minimo desiderato
    max_value = 10  # Valore massimo desiderato
    float_value = 1 + 9 * tf.sigmoid(x)
    return float_value

#output 
outputs = Dense(1, activation=custom_activation)(x)


#modello
model = Model(inputs=inputs, outputs=outputs, name='basic_model')

In [134]:
# definizione della loss e compilazione

from tensorflow.keras import optimizers

def weighted_loss(y_true, y_pred):
    weights=1+tf.exp((tf.abs(y_true-4)))
    loss=tf.reduce_mean(tf.square(y_true-y_pred)*weights)
    return loss

def macro_accuracy(y_true, y_pred):
    true_macroclass=tf.where(y_true < 4, tf.ones_like(y_true), tf.where(y_true < 7, 2 * tf.ones_like(y_true), 3 * tf.ones_like(y_true)))
    pred_macroclass=tf.where(y_pred < 4, tf.ones_like(y_pred), tf.where(y_pred < 7, 2 * tf.ones_like(y_pred), 3 * tf.ones_like(y_pred)))
    # Calcola l'accuratezza delle predizioni sulle macroclassi
    accuracy = tf.reduce_mean(tf.cast(tf.equal(true_macroclass, pred_macroclass), tf.float32))
    return accuracy

In [ ]:
from keras.callbacks import Callback
from tensorflow.keras import backend as K
from keras.optimizers import RMSprop, Adam

class LearningRateReducer(Callback):
    def __init__(self, initial_lr, reduce_factor, reduce_freq):
        super(LearningRateReducer, self).__init__()
        self.initial_lr = initial_lr
        self.reduce_factor = reduce_factor
        self.reduce_freq = reduce_freq

    def on_epoch_begin(self, epoch, logs=None):
        if epoch % self.reduce_freq == 0 and epoch != 0:
            lr = K.get_value(self.model.optimizer.lr)
            new_lr = lr * self.reduce_factor
            K.set_value(self.model.optimizer.lr, new_lr)
            print(f"Learning rate reduced to: {new_lr}")

# Definire i parametri
learning_rate = 0.001
reduce_factor = 0.5
reduce_freq = 2

# Creare l'istanza della callback personalizzata
lr_reducer = LearningRateReducer(initial_lr=learning_rate, reduce_factor=reduce_factor, reduce_freq=reduce_freq)


model.compile(
    loss='mean_squared_error',
    optimizer=optimizers.Adam(learning_rate=learning_rate),#optimizers.SGD(learning_rate=learning_rate),
    metrics=['mean_squared_error']
)


# Allenamento del modello con la callback personalizzata
history = model.fit(
    train_features,
    train_target,
    epochs=20,
    batch_size=32,
    validation_data=(val_features, val_target),
    verbose=1,
    callbacks=[lr_reducer]
)

In [ ]:
means.shape

In [ ]:
#evaluate model on the test set
test_loss, test_acc, test_mse = model.evaluate(features, means)

print()
print('Test Loss:\t', test_loss)
print('Test Accuracy:\t', test_acc)

In [ ]:
y_test_pred = model.predict(features)

In [ ]:
random_rows = np.random.choice(features.shape[0], size=10, replace=False)

import matplotlib.pyplot as plt

# visualizzare le prime 10 immagini e lable
fig, axs = plt.subplots(2,5,figsize=(24,9))

for i in np.arange(10):

    ax = axs.ravel()[i]
    idx=random_rows[i]
        
    ax.imshow(features[idx]/255, cmap='gray')
    ax.set_title('Mean: {} \n Class:{} '.format(np.round(means[idx], decimals=2),np.round(y_test_pred[idx],decimals=2)), fontsize=6)
    ax.axis('off')
    